# Paralelismo
Se refiere a utilizar varios procesos al mismo tiempo. Con julia se puede hacer en una sola computadora o distribuido. Para aprovechar los nucleos de procesamiento de tu computadora, es conveniente iniciar Julia un número equivalente de `workers` o trabajadores. En mi caso ese número es 4:
```shell
julia -p 4
```
Al igual que una co-rutina, para hacer funcionar a los trabajadores, hay que crear una llamada remota `remotecall()`. Ésta recive la función a processar, y sus argumentos:

In [2]:
addprocs(4)
r = remotecall(rand, 2, 2, 2)

Future(2, 1, 6, Nullable{Any}())

Para tomar un valor de ésta función, se debe mandar llamar a función `fetch()` con el trabajador de quien se quiere recolectar la información:

In [3]:
fetch(r)

2×2 Array{Float64,2}:
 0.776889  0.619507
 0.866355  0.326561

**NOTA** los trabajadores no tienen acceso al scope global, por lo que es necesario cargar las lbrerías para cáda worker.

In [4]:
function rand2(dims...)
    return 2*rand(dims...)
end
rand2(2,2)
fetch(@spawn rand2(2,2))

LoadError: [91mOn worker 2:
[91mUndefVarError: #rand2 not defined[39m
deserialize_datatype at .\serialize.jl:969
handle_deserialize at .\serialize.jl:674
deserialize at .\serialize.jl:634
handle_deserialize at .\serialize.jl:681
deserialize_global_from_main at .\distributed\clusterserialize.jl:153
foreach at .\abstractarray.jl:1731
deserialize at .\distributed\clusterserialize.jl:56
handle_deserialize at .\serialize.jl:723
deserialize at .\serialize.jl:634
handle_deserialize at .\serialize.jl:678
deserialize at .\serialize.jl:634
handle_deserialize at .\serialize.jl:681
deserialize_msg at .\distributed\messages.jl:98
message_handler_loop at .\distributed\process_messages.jl:161
process_tcp_streams at .\distributed\process_messages.jl:118
#99 at .\event.jl:73[39m

```shell
julia -p <n> -L módulo1.jl -L módulo2.jl MiPrograma.jl
```

# Simulación de montecarlo
Modelado a partir de eventos aleatorios.

In [5]:
# en Contar.jl
function contar(n)
    c::Int = 0
    for i=1:n
        c += rand(Bool)
    end
    c
end

contar (generic function with 1 method)

In [6]:
# En programa principal
@everywhere include("Contar.jl")

a = @spawn contar(100000000)
b = @spawn contar(100000000)
fetch(a)+fetch(b) # REDUCCIÓN!

LoadError: [91m[91mOn worker 2:
[91mSystemError: opening file C:\Users\Kelvyn\Dropbox\Curso JuliaLang\Simulation-of-Chemical-Processes-JuliaLang\Notebooks\01_Introducción - Día 1\Contar.jl: No such file or directory[39m
#systemerror#44 at .\error.jl:64 [inlined]
systemerror at .\error.jl:64
open at .\iostream.jl:104
open at .\iostream.jl:150
readstring at .\io.jl:541
#106 at .\distributed\process_messages.jl:268 [inlined]
run_work_thunk at .\distributed\process_messages.jl:56
macro expansion at .\distributed\process_messages.jl:268 [inlined]
#105 at .\event.jl:73[39m
#remotecall_fetch#141(::Array{Any,1}, ::Function, ::Function, ::Base.Distributed.Worker, ::Expr, ::Vararg{Expr,N} where N) at .\distributed\remotecall.jl:354
remotecall_fetch(::Function, ::Base.Distributed.Worker, ::Expr, ::Vararg{Expr,N} where N) at .\distributed\remotecall.jl:346
#remotecall_fetch#144(::Array{Any,1}, ::Function, ::Function, ::Int64, ::Expr, ::Vararg{Expr,N} where N) at .\distributed\remotecall.jl:36

Noten que estámos usando únicamente dos workers, cuando la mayoría de nosotros tiene por lo menos cuatro nucleos. El uso generalizado de workers se puede escribir con loops paralelos:

In [ ]:
n = @parallel (+) for i=1:200000000 # La reducción se lleva a cabo por la función +
  Int(rand(Bool))
end

# Mapeo en paralelo
Hablamos ya de la función `map()` y cómo ésta nos ayuda a mapear una función a un arreglo. Para utilizarla en paralelo, ya que la implementación a bajo nivel es ligeramente distinta, el nombre es `pmap()` (Parallel Maping). Esta función se encarga de organizar a los workers y evita que tengas que realizar llamadas remotas.

In [ ]:
M = Matrix{Float64}[rand(1000,1000) for i=1:10]
pmap(svd, M)

Eviten usar arreglos dentro de un ciclo for paralelizado, pues [normalmente](http://docs.julialang.org/en/release-0.5/manual/parallel-computing.html#shared-arrays) los arreglos no son compartidos entre trabajadores.

In [ ]:
a = SharedArray(Float64,10)
@parallel for i=1:10
  a[i] = i # NOT
end

# Clusters
Julia puede ser iniciado en paralelo con la opción `--machinefile` que lanzará un trabajador por cada linea en el archivo indicado. Las máquinas definidas en el archivo deben estár accesibles por medio de SSH sin contraseña, y tener julia instalado. El archivo de definición de máquinas debe tener los siguientes campos:
```shell
[count*][user@]host[:port] [bind_addr[:port]] 
```
Dónde:
- `user` tiene por default el mismo usuario que el acutal
- `port` es el puerto default de SSH (22)
- `count` es el número de workers a trabajar en el nódo (default 1)
- La opción `bind_addr[:port]` especifica la ip y puerto a la que otros workers deben entrar para conectarse con el actual.

A darle! :)
